# Install transformers and import libraries

In [1]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   glue_convert_examples_to_features
)
import tensorflow as tf
tf.config.experimental.list_physical_devices('CPU')
import tensorflow_datasets as tfds
import json

# Choose a Model from the Huggingface Library

In [2]:
# Choose model
# @markdown >The default model is <i><b>COVID-Twitter-BERT</b></i>. You can however choose <i><b>BERT Base</i></b> or <i><b>BERT Large</i></b> to compare these models to the <i><b>COVID-Twitter-BERT</i></b>. All these three models will be initiated with a random classification layer. If you go directly to the Predict-cell after having compiled the model, you will see that it still runs the predition. However the output will be random. The training steps below will finetune this for the specific task. <br /><br /> 
model_name = 'digitalepidemiologylab/covid-twitter-bert' #@param ["digitalepidemiologylab/covid-twitter-bert", "bert-large-uncased", "bert-base-uncased"]

# Initialise tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Download the SST-2 Dataset and Prepare for Finetuning
You can skip this step if you are using the already finetuned model

In [3]:
# Paramteters
#@markdown >Batch size and sequence length needs to be set to prepare the data. The size of the batches depends on available memory. For Colab GPU limit batch size to 8 and sequence length to 96. By reducing the length of the input (max_seq_length) you can also increase the batch size. For a dataset like SST-2 with lots of short sentences. this will likely benefit training.
max_seq_length = 96 #@param {type: "integer"}
train_batch_size =  8 #@param {type: "integer"} 
eval_batch_size = 8 #@param {type: "integer"}


#@markdown >The Glue dataset has around 62000 examples, and we really do not need them all for training a decent model. To cut down training time, please reduse this to only a percentage of the entire set.
use_percentage_of_data = 5 #@param {type: "slider", min: 1, max: 100}

# get dataset sizes
glue_builder = tfds.builder('glue/sst2')
num_train_examples = glue_builder.info.splits['train'].num_examples
num_dev_examples = glue_builder.info.splits['validation'].num_examples
num_labels = glue_builder.info.features['label'].num_classes

# download datasets and convert to training features
#glue_builder.download_and_prepare()
train_data = glue_builder.as_dataset(split='train')
train_dataset = glue_convert_examples_to_features(train_data, tokenizer, max_length=max_seq_length, task='sst-2')
train_dataset = train_dataset.shuffle(100).batch(train_batch_size)

dev_data = glue_builder.as_dataset(split='validation')
dev_dataset = glue_convert_examples_to_features(dev_data, tokenizer, max_length=max_seq_length, task='sst-2')
dev_dataset = dev_dataset.shuffle(100).batch(eval_batch_size)

# Map the labels for printing
label_mapping = {i: glue_builder.info.features['label'].int2str(i) for i in range(num_labels)}

print(f'\n\nThe dataset is downloaded. The entire dataset has {num_train_examples + num_dev_examples} examples of which you are using {use_percentage_of_data}%. This will result in a train dataset with {int(num_train_examples * (use_percentage_of_data/100))} examples and a validation dataset with {int(num_dev_examples * (use_percentage_of_data/100))} examples.')

INFO:absl:Load dataset info from C:\Users\Eugene\tensorflow_datasets\glue\sst2\1.0.0
INFO:absl:Constructing tf.data.Dataset for split train, from C:\Users\Eugene\tensorflow_datasets\glue\sst2\1.0.0
INFO:absl:Constructing tf.data.Dataset for split validation, from C:\Users\Eugene\tensorflow_datasets\glue\sst2\1.0.0




The dataset is downloaded. The entire dataset has 68221 examples of which you are using 5%. This will result in a train dataset with 3367 examples and a validation dataset with 43 examples.


# Compile the Model, Train it on the SST-2 Task and Save the Result
You can skip this step if you are using the already finetuned model

In [4]:
#@markdown >The default learning rate of 2e5 will be fine in most cases
learning_rate = 2e-5 #@param {type: "number"}

#@markdown > Typically these type of models are finetuned for 3 epochs. This can be increased for small datasets and decreased for large datasets.
num_epochs = 1  #@param {type: "integer"}

with tf.device('/cpu:0'):
    # Initialise a Model for Sequence Classification with 2 labels
    config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

    # Optimizer and loss
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Metrics and callbacks
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
    checkpoint_path = './checkpoints/checkpoint.{epoch:02d}'
    callbacks = [tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True)]

    # Compute some variables
    train_steps_per_epoch = int(num_train_examples * (use_percentage_of_data/100) / train_batch_size)
    dev_steps_per_epoch = int(num_dev_examples * (use_percentage_of_data/100) / eval_batch_size)

    # Compile model
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    # Train the model
    history = model.fit(train_dataset, 
      epochs=num_epochs,
      steps_per_epoch=train_steps_per_epoch,
      validation_data=dev_dataset,
      validation_steps=dev_steps_per_epoch,
      callbacks=callbacks)

# Print some information about the training
print(f'\nThe training has finished training after {num_epochs} epochs.')
print('\nThe history contains the accuracy and loss at every epoch:')
print(json.dumps(history.history, indent=4))

model.save_pretrained('./huggingface_model/')

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


420/420 [==============================] - 2899s 7s/step - loss: 0.3553 - accuracy: 0.8536 - val_loss: 0.1524 - val_accuracy: 0.9500

The training has finished training after 1 epochs.

The history contains the accuracy and loss at every epoch:
{
    "loss": [
        0.35534483194351196
    ],
    "accuracy": [
        0.8535714149475098
    ],
    "val_loss": [
        0.152359277009964
    ],
    "val_accuracy": [
        0.949999988079071
    ]
}


# Predict

In [5]:
# Small function only used for formatting the output
def format_prediction(preds, label_mapping, label_name):
    preds = tf.nn.softmax(preds, axis=1)
    formatted_preds = []
    for pred in preds.numpy():
        # convert to Python types and sort
        pred = {label: float(probability) for label, probability in zip(label_mapping.values(), pred)}
        pred = {k: v for k, v in sorted(pred.items(), key=lambda item: item[1], reverse=True)}
        formatted_preds.append({label_name: list(pred.keys())[0], f'{label_name}_probabilities': pred})
    return formatted_preds

In [6]:
#@markdown >Please input text that the model can try to classify
input_text = "none of this violates the letter of book"  #@param {type: "string"}

# Tokenize the input 
input_ids = tf.constant(tokenizer.encode(input_text, add_special_tokens=True))[None, :]

# Run predictions
preds = model(input_ids)

# format logits
formatted_preds = format_prediction(preds[0], label_mapping, 'sentiment')

print(f'\nLabel Mapping:{json.dumps(label_mapping, indent=4)}')
print(f'\nLogits: {preds}')
print(f'\nProbabilities:{json.dumps(formatted_preds, indent=4)}')


Label Mapping:{
    "0": "negative",
    "1": "positive"
}

Logits: (<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.1184359 ,  0.79018664]], dtype=float32)>,)

Probabilities:[
    {
        "sentiment": "positive",
        "sentiment_probabilities": {
            "positive": 0.7127182483673096,
            "negative": 0.2872817814350128
        }
    }
]


In [7]:
#@markdown >Please input text that the model can try to classify
input_text = "while the mystery surrounding the nature of the boat 's malediction remains intriguing enough to sustain mild interest , the picture refuses to offer much accompanying sustenance in the way of characterization , humor or plain old popcorn fun ."  #@param {type: "string"}

# Tokenize the input 
input_ids = tf.constant(tokenizer.encode(input_text, add_special_tokens=True))[None, :]

# Run predictions
preds = model(input_ids)

# format logits
formatted_preds = format_prediction(preds[0], label_mapping, 'sentiment')

print(f'\nLabel Mapping:{json.dumps(label_mapping, indent=4)}')
print(f'\nLogits: {preds}')
print(f'\nProbabilities:{json.dumps(formatted_preds, indent=4)}')


Label Mapping:{
    "0": "negative",
    "1": "positive"
}

Logits: (<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.4414123, -1.6009173]], dtype=float32)>,)

Probabilities:[
    {
        "sentiment": "negative",
        "sentiment_probabilities": {
            "negative": 0.954450249671936,
            "positive": 0.045549776405096054
        }
    }
]


In [8]:
#@markdown >Please input text that the model can try to classify
input_text = "gangs of new york is an unapologetic mess , whose only saving grace is that it ends by blowing just about everything up ."  #@param {type: "string"}

# Tokenize the input 
input_ids = tf.constant(tokenizer.encode(input_text, add_special_tokens=True))[None, :]

# Run predictions
preds = model(input_ids)

# format logits
formatted_preds = format_prediction(preds[0], label_mapping, 'sentiment')

print(f'\nLabel Mapping:{json.dumps(label_mapping, indent=4)}')
print(f'\nLogits: {preds}')
print(f'\nProbabilities:{json.dumps(formatted_preds, indent=4)}')


Label Mapping:{
    "0": "negative",
    "1": "positive"
}

Logits: (<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.5006818, -1.549777 ]], dtype=float32)>,)

Probabilities:[
    {
        "sentiment": "negative",
        "sentiment_probabilities": {
            "negative": 0.9548023343086243,
            "positive": 0.045197661966085434
        }
    }
]
